In [15]:
%matplotlib inline
import numpy as np
import pandas as pd
import re as re

import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit


# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.callbacks import History,LearningRateScheduler
# from tensorflow.keras.layers import Dropout
# print('TensorFlow %s, Keras %s, numpy %s, pandas %s'%(tf.__version__,keras.__version__, np.__version__,pd.__version__))
__DEBUG__=False

In [ ]:
# La fonction pandas pd.read_csv permet de créer un objet Dataframe à partir d'un csv

# Données avec labels
train = pd.read_csv('data_concatener.csv', sep=';')
# Données de tests sans label. Les prédictions de survie seront envoyées à kaggle
test  = pd.read_csv('Test_set_values.csv')
# On réunit les données dans une liste (pour pouvoir boucler sur les 2 dataframes)
full_data = [train, test]
#On garde les passagers ID des données test, car on en aura besoin pour le fichiers résultats de kaggle (voir l'exemple gender_submission.csv)
finalfile_index=test.id #Index des données de test pour le résultat final

for dataset in full_data:
    dataset.loc[dataset.public_meeting.isnull(), 'public_meeting'] = False

#La fonction info() permet de répérer les colonnes avec des valeurs nulles
print('\nTrain data:')
train.info()
print('\nTest data:')
test.info()
print('Dashboard:')
print(train['public_meeting'])
train.to_excel('output_final.xlsx')


Train data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 41 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null object
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           59400 non-n